# Projektplanung als Ganzzahlprogramm

Ein Projektplanungsproblem als Ganzzahlprogramm modelliert, siehe Abschnitt 3.7)

In [ ]:
# Zum Installieren in google colab, entfernen Sie die Raute in der nächsten Zeile
#!pip install mip

In [1]:
import random
import mip

Wir definieren eine Methode, um Zufallsinstanzen zu erzeugen. Neben der Anzahl der Projekte und der Anzahl der Berater wird noch eine Wahrscheinlichkeit übergeben. Bei jedem Berater:in wird zufällig bestimmt, ob er oder sie die nötige Kompetenz für das jeweilige Projekt hat. 

In [2]:
def random_instance(num_consultants,num_projects,probability):
    profits=[random.randint(10,50)*1000 for _ in range(num_projects)]
    competences=[[0 for _ in range(num_projects)] for _ in range(num_consultants)]
    for a in range(num_consultants):
        for b in range(num_projects):
            if random.random()<=probability:
                competences[a][b]=1
    return profits,competences

random_instance(3,4,0.5)

([41000, 37000, 36000, 18000], [[1, 0, 0, 1], [1, 1, 0, 1], [0, 0, 1, 0]])

Die Instanz besteht also einem Profitvektor und einer Matrix. Der Profitvektor gibt an, welcher Gewinn erzielt wird, wenn das jeweilige Projekt angenommen wird. Die Matrix gibt an, ob die Berater:innen für das Projekt geeignet sind.

In [3]:
num_consultants=13
num_projects=6

p,c=random_instance(num_consultants,num_projects,0.5)

Wir definieren als erstes das Modell und fügen die Variablen $y_b\in\{0,1\}$ und $x_{ab}\in\{0,1\}$ hinzu:

In [4]:
m=mip.Model()
y=[m.add_var(var_type=mip.BINARY) for _ in range(num_projects)]
x=[[m.add_var(var_type=mip.BINARY) for _ in range(num_projects)] for _ in range(num_consultants)]

Das Ziel ist die Maximierung des Profits:

$\max \sum_{b\in B} y_bp_b$

In [5]:
m.objective=mip.maximize(mip.xsum(y[b]*p[b] for b in range(num_projects)))

Berater:innen dürfen nur in einem Projekt eingesetzt werden, für das sie die nötigen Kompetenzen mitbringen:

$x_{ab}\leq c_{ab}\quad\text{für alle }a\in A,b\in B$

In [6]:
for a in range(num_consultants):
    for b in range(num_projects):
        m+=x[a][b]<=c[a][b]

Berater:innen können nur in einem Projekt mitarbeiten:

$\sum_{b\in B}x_{ab}\leq 1\quad\text{für alle }a\in A$

In [7]:
for a in range(num_consultants):
    m+=mip.xsum(x[a][b] for b in range(num_projects))<=1

Jedes angenommene Projekt benötigt (mindestens) drei Berater:innen:

$\sum_{a\in A}x_{ab}\geq 3y_b\quad\text{für alle }b\in B$

In [8]:
for b in range(num_projects):
    m+=mip.xsum(x[a][b] for a in range(num_consultants))>=3*y[b]

Wir starten die Optimierung:

In [9]:
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

...und geben das Ergebnis aus:

In [10]:
realised=[b for b in range(num_projects) if y[b].x==1]
print("Angenommene Projekte:")
for b in realised:
    print("Projekt {}".format(b))
    consultants=[a for a in range(num_consultants) if x[a][b].x==1]
    print("  Berater: {}".format(consultants))
    print("  Profit : {}".format(p[b]))
not_realised=[b for b in range(num_projects) if y[b].x==0]
print("Abgelehnte Projekte:")
for b in not_realised:
    print("  Projekt {} mit Profit {}".format(b,p[b]))
print("------")
print("Gesamtprofit: {}".format(m.objective_value))

Angenommene Projekte:
Projekt 1
  Berater: [0, 9, 10]
  Profit : 36000
Projekt 2
  Berater: [6, 7, 11]
  Profit : 24000
Projekt 4
  Berater: [1, 4, 8]
  Profit : 30000
Projekt 5
  Berater: [2, 3, 12]
  Profit : 34000
Abgelehnte Projekte:
  Projekt 0 mit Profit 17000
  Projekt 3 mit Profit 12000
------
Gesamtprofit: 124000.0
